In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
# Getting all the response from the main page
response = requests.get('https://vedabase.io/en/library/bg/')
if response.status_code == 200:
    print("yes")

yes


In [ ]:
# fliter out the links for the main pages/chapters
# print(response.content)
soup = BeautifulSoup(response.content, 'html.parser')
print(soup.prettify())

In [61]:
# function to assign data to the dictionary

def assign_to_dict(shloka_num, shloka_dev, shloka_en, shloka_translation, shloka_purport):
    my_dict = {}
    print(shloka_num.split("."))
    # generating chapter num
    chapter = shloka_num.split(".")[1].rsplit()[0]
    shloka = shloka_num.split(".")[2]

    my_dict = {'chapter' : chapter, 'shloka' : shloka, 'sanskrit' : shloka_dev,
               'english': shloka_en, 'translation' : shloka_translation, 
              'purport': shloka_purport}
    return my_dict
    

In [62]:
sources = soup.find_all('a')
chapter_links = []
shlokas_links = []
shlokas_dict_list = []
dict_list = []
# ctr = 0 
# link to the chapters
for source in sources:
    if "/en/library/bg/" in source['href'] and "Chapter" in source.text: # this is kinda static condition
        
        # print(source)
        url = "https://www.vedabase.io" + source['href']
        # print(url)
        
        # trying to check the status for the shlokas
        print(f"checking response for chepter: {url.split('/')[-2]}")
        
        # link to the verses
        response = requests.get(url)
        
        # extracting each verse l
        verse_soup = BeautifulSoup(response.content, "html.parser")
        # print(verse_soup.prettify())

        # getting verses links
        verses_links = verse_soup.find_all('a')
        # iterate over the links that contains the links to the shloka
        ctr = 0
        for verse in verses_links:
            if "TEXT" in verse.text:

                # generate the url to get the content from each shloka
                verse_url = "https://vedabase.io" + verse['href'] 
                resp = requests.get(verse_url)
                shloka_soup = BeautifulSoup(resp.content, "html.parser")
                ctr+=1
                # if ctr == 4:
                    # print(shloka_soup.find(id='content'))
                for br in shloka_soup.find_all('br'):
                    br.replace_with('\n')
                
                
                # get the whole section of shloka, meaning and purport
                shloka_num = shloka_soup.find(id='content').find('h1').text
                shloka_dev = shloka_soup.find(class_ = 'r r-devanagari').text
                shloka_en = shloka_soup.find(class_ = 'r r-lang-en r-verse-text').text
                shloka_translation = shloka_soup.find(class_ = 'r r-lang-en r-translation').text

                # assign first 
                # sometimes there is no purport for very simple and factual shlokas in that case assign none 
                # shloka_purport = shloka_soup.find(class_ ='r r-lang-en r-paragraph')
                # print(repr(shloka_dev))
                try:
                    shloka_purport = shloka_soup.find(class_ ='r r-lang-en r-paragraph').text
                    
                except:
                    print("No purport")
                    shloka_purport = None
                # print(verse_url)
                # this returns the dictionary
                data = assign_to_dict(shloka_num, shloka_dev, shloka_en, shloka_translation, shloka_purport)
                dict_list.append(data)

                # print(shloka_purport, "\n")
                # print(f"{shloka_num} \n{shloka_dev} {shloka_en} {shloka_translation} {shloka_purport}")
                
                
               
        
  
            
                            
    

checking response for chepter: 1
'धृतराष्ट्र उवाच\nधर्मक्षेत्रे कुरुक्षेत्रे समवेता युयुत्सव: ।\nमामका: पाण्डवाश्चैव किमकुर्वत सञ्जय ॥ १ ॥'
['Bg', ' 1', '1']
'सञ्जय उवाच\nदृष्ट्वा तु पाण्डवानीकं व्यूढं दुर्योधनस्तदा ।\nआचार्यमुपसङ्गम्य राजा वचनमब्रवीत् ॥ २ ॥'
['Bg', ' 1', '2']
'पश्यैतां पाण्डुपुत्राणामाचार्य महतीं चमूम् ।\nव्यूढां द्रुपदपुत्रेण तव शिष्येण धीमता ॥ ३ ॥'
['Bg', ' 1', '3']
'अत्र श\u200dूरा महेष्वासा भीमार्जुनसमा युधि ।\nयुयुधानो विराटश्च द्रुपदश्च महारथः ॥ ४ ॥'
['Bg', ' 1', '4']
'धृष्टकेतुश्चेकितानः काशिराजश्च वीर्यवान् ।\nपुरुजित्कुन्तिभोजश्च शैब्यश्च नरपुङ्गवः ॥ ५ ॥'
No purport
['Bg', ' 1', '5']
'युधामन्युश्च विक्रान्त उत्तमौजाश्च वीर्यवान् ।\nसौभद्रो द्रौपदेयाश्च सर्व एव महारथाः ॥ ६ ॥'
No purport
['Bg', ' 1', '6']
'अस्माकं तु विशिष्टा ये तान्निबोध द्विजोत्तम ।\nनायका मम सैन्यस्य संज्ञार्थ तान्ब्रवीमि ते ॥ ७ ॥'
No purport
['Bg', ' 1', '7']
'भवान्भीष्मश्च कर्णश्च कृपश्च समितिंजयः ।\nअश्वत्थामा विकर्णश्च सौमदत्तिस्तथैव च ॥ ८ ॥'
['Bg', ' 1', '8']
'अन्ये च बहवः श\u200dूरा मद

In [57]:
# now converting the list of dictionaries to the json
import json

final_json = json.dumps(dict_list,  ensure_ascii=False, indent = 2)
print(final_json)

[
  {
    "chapter": "1",
    "shloka": "1",
    "sanskrit": "धृतराष्ट्र उवाच\nधर्मक्षेत्रे कुरुक्षेत्रे समवेता युयुत्सव: ।\nमामका: पाण्डवाश्चैव किमकुर्वत सञ्जय ॥ १ ॥",
    "english": "dhṛtarāṣṭra uvāca\ndharma-kṣetre kuru-kṣetre\nsamavetā yuyutsavaḥ\nmāmakāḥ pāṇḍavāś caiva\nkim akurvata sañjaya",
    "translation": "Dhṛtarāṣṭra said: O Sañjaya, after my sons and the sons of Pāṇḍu assembled in the place of pilgrimage at Kurukṣetra, desiring to fight, what did they do?",
    "purport": "Bhagavad-gītā is the widely read theistic science summarized in the Gītā-māhātmya (Glorification of the Gītā). There it says that one should read Bhagavad-gītā very scrutinizingly with the help of a person who is a devotee of Śrī Kṛṣṇa and try to understand it without personally motivated interpretations. The example of clear understanding is there in the Bhagavad-gītā itself, in the way the teaching is understood by Arjuna, who heard the Gītā directly from the Lord. If someone is fortunate enough to und

In [63]:
# now dumping it
with open("bhagavad_gita.json", "w", encoding="utf-8") as json_file:
    json.dump(dict_list, json_file, ensure_ascii=False, indent=4)

In [ ]:
# checking fetching urls from each chapter
